In [2]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

In [3]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

In [6]:
class SimpleNet(torch.nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        #self.hidden = torch.nn.Linear(784,548)   # hidden layer
        #self.out = torch.nn.Linear(548,10)   # output layer
        self.fc1 = nn.Linear(784, 576)
        self.bc1 = nn.BatchNorm1d(576)
        
        self.fc2 = nn.Linear(576, 324)
        self.bc2 = nn.BatchNorm1d(324)
        
        self.fc3 = nn.Linear(324, 144)
        self.bc3 = nn.BatchNorm1d(144)
        
        self.fc4 = nn.Linear(144, 10)

    def forward(self, x):
        #x = F.relu(self.hidden(x))      # activation function for hidden layer
        #x = self.out(x)
        #return x
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = self.bc1(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.5, training=self.training)#防止出现过拟合
        
        h = self.fc2(h)
        h = self.bc2(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.2, training=self.training)#防止出现过拟合
        
        h = self.fc3(h)
        h = self.bc3(h)
        h = nn.functional.relu(h)
        h = nn.functional.dropout(h, p=0.1, training=self.training)#防止出现过拟合
        
        h = self.fc4(h)
        out = nn.functional.log_softmax(h, dim = 0)
        return out
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.CrossEntropyLoss()
optimizer =torch.optim.Adam(model.parameters(), lr=0.0001)

In [7]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    train_acc = 0
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        #images = Variable(images)          #包装tensor用于自动求梯度
        #labels = Variable(labels)
        optimizer.zero_grad()       #优化器梯度归零
        out = model(images)           #正向传播
        lossvalue = criterion(out,labels)        #求损失值
        optimizer.zero_grad()       #优化器梯度归零
        lossvalue.backward()    #反向转播，刷新梯度值
        optimizer.step()        #优化器运行一步，注意optimizer搜集的是model的参数

        #计算损失
        train_loss += float(lossvalue)      
      #计算精确度
        _,pred = out.max(1)
        num_correct = (pred == labels).sum()
        acc = int(num_correct) / images.shape[0]
        train_acc += acc
        #train_acc += (torch.max(out, 1)[1].view(labels.size()).data == labels.data).sum()
        
              
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    eval_loss = 0
    eval_acc = 0
    model.eval() #模型转化为评估模式
    for images,labels in tqdm(test_loader):
        images = images.view(-1,784)
        #images = Variable(images)
        #label = Variable(label)
        testout = model(images)
        testloss = criterion(testout,labels)
        eval_loss += float(testloss)
 
        _,pred = testout.max(1)
        num_correct = (pred == labels).sum()
        acc = int(num_correct) / images.shape[0]
        eval_acc += acc
        #train_acc += (torch.max(testout, 1)[1].view(labels0.size()).data == labels0.data).sum()
        
    train_loss=train_loss/len(train_loader)
    train_acc=train_acc/len(train_loader)
    eval_loss=eval_loss/len(test_loader)
    eval_acc=eval_acc/len(test_loader)
    print("[Epoch: %d] Train Loss: %5.5f Train Accuracy: %5.5f" % (epoch+1, train_loss, train_acc))
    print("[Epoch: %d] Test Loss: %5.5f Test Accuracy: %5.5f" % (epoch+1, eval_loss, eval_acc))
    

  0%|▎                                                                                 | 2/468 [00:00<00:25, 18.57it/s]

[Epoch: 1] Train Loss: 0.78364 Train Accuracy: 0.81824
[Epoch: 1] Test Loss: 0.27280 Test Accuracy: 0.93610


  0%|▎                                                                                 | 2/468 [00:00<00:26, 17.75it/s]

[Epoch: 2] Train Loss: 0.20960 Train Accuracy: 0.93657
[Epoch: 2] Test Loss: 0.14963 Test Accuracy: 0.95453


  0%|▎                                                                                 | 2/468 [00:00<00:26, 17.91it/s]

...

[Epoch: 8] Train Loss: 0.04757 Train Accuracy: 0.98499
[Epoch: 8] Test Loss: 0.06698 Test Accuracy: 0.97987


  0%|▎                                                                                 | 2/468 [00:00<00:24, 19.10it/s]

[Epoch: 9] Train Loss: 0.04062 Train Accuracy: 0.98726
[Epoch: 9] Test Loss: 0.06779 Test Accuracy: 0.98007


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 39.96it/s]

[Epoch: 10] Train Loss: 0.03458 Train Accuracy: 0.98877
[Epoch: 10] Test Loss: 0.06254 Test Accuracy: 0.98087


In [11]:
print('Training Accuracy: %.2f%%' % (train_acc*100))
print('Testing Accuracy: %.2f%%' % (eval_acc*100))

Training Accuracy: 98.88%
Testing Accuracy: 98.09%
